[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Mustapha-AJEGHRIR/medical_txt_parser/blob/main/src/notebooks/assertions_nli/ast_nli_scibert.ipynb)

# Assertions classification

Based of: https://github.com/huggingface/transformers/blob/master/examples/pytorch/text-classification/run_xnli.py

In [2]:
!nvidia-smi

Sat Jan 22 13:00:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [17]:
%%capture
!pip install seqeval transformers datasets spacy sentence_transformers

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/projects/medical_txt_parser

ModuleNotFoundError: No module named 'google.colab'

In [155]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

path = %pwd
while "src" in path:
    %cd ..
    path = %pwd

import glob
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import random

import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import datasets
import numpy as np
from datasets import load_dataset, load_metric , Dataset
import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version
from transformers import pipeline

require_version("datasets>=1.8.0", "To fix: pip install --upgrade datasets")

from src.utils.parse_data import parse_ast, parse_concept, parse_relation

In [11]:
train_data_path = "data/train"
val_data_path = "data/val"
ast_folder_name = "ast"
concept_folder_name = "concept"
rel_folder_name = "rel"
txt_folder_name = "txt"
nli_data_path = "data/nli"
# os.makedirs(nli_data_path, exist_ok=True)

# model args
model_name_or_path = "gsarti/scibert-nli"
cache_dir = None
model_revision = None 
tokenizer_name = model_name_or_path
do_lower_case = True
use_fast_tokenizer = True
fp16 = True

# data args
pad_to_max_length = None
max_seq_length = None

set_seed(42)


### Import data

In [5]:
text_files = glob.glob(train_data_path + os.sep + txt_folder_name + os.sep +  "*.txt")
filename = ""
df = pd.DataFrame()
for file in tqdm(text_files):
    with open(file, 'r') as f:
        text = f.read()
        filename = file.split("/")[-1].split(".")[0]
        ast = parse_ast(train_data_path + os.sep + ast_folder_name + os.sep +  filename + ".ast")
        
        df = df.append(pd.DataFrame({"text": [text], "filename": [filename], "ast":[ast]}), ignore_index=True)
df.head()

100%|██████████| 170/170 [00:00<00:00, 537.99it/s]


,text,filename,ast
0,143748600 SC\n43243309\n095342\n09/30/2000 12:...,143748600_SC,"{'concept_text': ['htn', 'atypical chest pain'..."
1,Admission Date :\n2014-03-31\nDischarge Date :...,record-56,"{'concept_text': ['sclera mildly icteric', 'ac..."
2,699905656 SC\n06100044\n921737\n1/17/1993 12:0...,699905656_SC,{'concept_text': ['a lateral l-1 s-1 disc bulg...
3,627258104\nGH\n56900479\n1/8/2001 12:00:00 AM\...,627258104,"{'concept_text': ['bradycardia', 'subsequent f..."
4,839999049 YC\n59242403\n431924\n11/01/2000 12:...,839999049_YC,"{'concept_text': ['pauses', 'ischemia', 'htn',..."


In [123]:
assertion_df = pd.DataFrame(columns=[ "filename"]+list(ast.keys()))
for i, file in df.iterrows():
    assertion_dict = file["ast"]
    tmp = pd.DataFrame(assertion_dict)
    tmp["filename"] = file["filename"]
    assertion_df = assertion_df.append(tmp, ignore_index=True)
assertion_df.head()

,filename,concept_text,start_line,start_word_number,end_line,end_word_number,concept_type,assertion_type
0,143748600_SC,htn,52,19,52,19,problem,present
1,143748600_SC,atypical chest pain,5,0,5,2,problem,present
2,143748600_SC,dissection,56,7,56,7,problem,possible
3,143748600_SC,anxiety ( anxiety ),48,18,48,21,problem,present
4,143748600_SC,back pain,56,11,56,12,problem,present


### Dataset Preprocessing

In [124]:
def preprocess_text(row):
    text = row["text"]
    # find line
    text = text.lower()
    text = text.split("\n")[row["start_line"]-1]
    row["text"] = text
    return row
ast2label = {ast:i for i,ast in enumerate(assertion_df["assertion_type"].unique())}
label2ast = {i:ast for ast,i in ast2label.items()}

assertion_df = assertion_df.merge(df[["filename","text"]], on="filename", how="inner")
assertion_df = assertion_df.apply(preprocess_text, axis=1)
assertion_df = assertion_df.drop(columns=["filename", "start_word_number", "end_line", "end_word_number", "concept_type", "start_line"])
assertion_df["label"] = assertion_df["assertion_type"].apply(lambda x: ast2label[x])
assertion_df = assertion_df.rename(columns={"text": "sentence1", "concept_text": "sentence2"})
assertion_df = assertion_df[["sentence1", "sentence2", "label", "assertion_type"]]
assertion_df


,sentence1,sentence2,label,assertion_type
0,"41 yo man with crfs of dm type ii , high chole...",htn,0,present
1,atypical chest pain,atypical chest pain,0,present
2,had chest ct to r / o dissection ( due to back...,dissection,1,possible
3,"other diagnosis ; conditions , infections , co...",anxiety ( anxiety ),0,present
4,had chest ct to r / o dissection ( due to back...,back pain,0,present
...,...,...,...,...
7068,specimens sent to pathology included peritonea...,falciform nodules,0,present
7069,she was uncomfortable and slow to have bowel m...,uncomfortable,0,present
7070,is significant for spinal stenosis and a serie...,a ureteral stricture,0,present
7071,"ms. sterpkoteneish is a 69-year-old woman , wh...",increasing pain in her left upper quadrant,0,present


In [132]:
# Build HuggingFace Dataset

train_df, val_df = train_test_split(assertion_df, train_size=None, shuffle=True, test_size=100, stratify=assertion_df["label"], random_state=42)

features = datasets.Features({'sentence1': datasets.Value(dtype='string'),
 'sentence2': datasets.Value(dtype='string'),
 'label': datasets.ClassLabel(num_classes=len(label2ast), names=list(label2ast.values())),
 'assertion_type': datasets.Value(dtype='string')})

train_dataset = Dataset.from_pandas(train_df, preserve_index=False, features=features)
eval_dataset = Dataset.from_pandas(val_df, preserve_index=False, features=features)

label_list = train_dataset.features["label"].names
num_labels = len(label_list)

label_list = train_dataset.features["label"].names
train_dataset, eval_dataset

(Dataset({
     features: ['sentence1', 'sentence2', 'label', 'assertion_type'],
     num_rows: 6973
 }),
 Dataset({
     features: ['sentence1', 'sentence2', 'label', 'assertion_type'],
     num_rows: 100
 }))

In [133]:
# check labels balance
print(f"train_df labels: {train_df['label'].value_counts()}")
print(f"val_df labels: {val_df['label'].value_counts()}")

train_df labels: 0    4559
2    1573
4     376
1     305
5      88
3      72
Name: label, dtype: int64
val_df labels: 0    65
2    23
4     6
1     4
3     1
5     1
Name: label, dtype: int64


In [134]:
# Load pretrained model and tokenizer
# In distributed training, the .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    finetuning_task="xnli",
    cache_dir=cache_dir,
    revision=model_revision,
    label2id=ast2label,
    id2label=label2ast
)
tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_name if tokenizer_name else model_name_or_path,
    do_lower_case=do_lower_case,
    cache_dir=cache_dir,
    use_fast=use_fast_tokenizer,
    revision=model_revision,
)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name_or_path,
    from_tf=bool(".ckpt" in model_name_or_path),
    config=config,
    cache_dir=cache_dir,
    revision=model_revision,
)

loading configuration file https://huggingface.co/gsarti/scibert-nli/resolve/main/config.json from cache at /home/jupyter/.cache/huggingface/transformers/d1a718a92956f71237b03954f057420eda133c6f2f5980416169abae733cd0c5.3e1114112ea2a7e54342079530b1a728c8757051c6ab0d54ff742f9f92c549a4
Model config BertConfig {
  "_name_or_path": "gsarti/scibert-nli",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "finetuning_task": "xnli",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "present",
    "1": "possible",
    "2": "absent",
    "3": "conditional",
    "4": "hypothetical",
    "5": "associated_with_someone_else"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "absent": 2,
    "associated_with_someone_else": 5,
    "conditional": 3,
    "hypothetical": 4,
    "possible": 1,
    "present": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embe

In [135]:
# Preprocessing the datasets
# Padding strategy
if pad_to_max_length:
    padding = "max_length"
else:
    # We will pad later, dynamically at batch creation, to the max sequence length in each batch
    padding = False


def preprocess_function(examples):
    # Tokenize the texts
    return tokenizer(
        examples["sentence1"],
        examples["sentence2"],
        padding=padding,
        max_length=max_seq_length,
        truncation=True,
    )


train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    desc="Running tokenizer on train dataset",
)
eval_dataset = eval_dataset.map(
    preprocess_function,
    batched=True,
    desc="Running tokenizer on validation dataset",
)

# Log a few random samples from the training set:
for index in random.sample(range(len(train_dataset)), 3):
    print(f"Sample {index} of the training set: {train_dataset[index]}.\n")


Running tokenizer on train dataset:   0%|          | 0/7 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Running tokenizer on validation dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Sample 5238 of the training set: {'sentence1': '70 y/o male with h/o cad , t2dm , hyperlipidemia , and copd who presented to the juan on 2017-09-03 with worsening sob x 5 days from morton hospital & medical center .', 'sentence2': 't2dm', 'label': 0, 'assertion_type': 'present', 'input_ids': [101, 3599, 411, 1338, 114, 3859, 188, 160, 1338, 114, 18144, 430, 104, 30130, 19294, 430, 2071, 24259, 430, 136, 2930, 30116, 1010, 1642, 146, 111, 8656, 132, 201, 4754, 578, 13947, 578, 12197, 188, 25308, 669, 30125, 631, 312, 2015, 277, 3359, 109, 3561, 908, 3086, 4001, 211, 102, 104, 30130, 19294, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}.

Sample 912 of the training set: 

In [136]:
# Get the metric function
f1_metric = load_metric("f1")
precision_metric = load_metric("precision")
recall_metric = load_metric("recall")
accuracy_metric = load_metric("accuracy")

# You can define your custom compute_metrics function. It takes an `EvalPrediction` object (a namedtuple with a
# predictions and label_ids field) and has to return a dictionary string to float.
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)

    metrics = {}
    metrics.update(f1_metric.compute(predictions=preds, references=p.label_ids, average="macro"))
    metrics.update(precision_metric.compute(predictions=preds, references=p.label_ids, average="macro"))
    metrics.update(recall_metric.compute(predictions=preds, references=p.label_ids, average="macro"))
    metrics.update(accuracy_metric.compute(predictions=preds, references=p.label_ids))
    return metrics

In [137]:
# Data collator will default to DataCollatorWithPadding, so we change it if we already did the padding.
if pad_to_max_length:
    data_collator = default_data_collator
elif fp16:
    data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)
else:
    data_collator = None

In [138]:
model_name_or_path

'gsarti/scibert-nli'

In [317]:
# Initialize our Trainer
model_folder_name = f"{model_name_or_path.split('/')[-1]}-ast-clf-1"
args = TrainingArguments(
    f"training_logs/{model_folder_name}",
    evaluation_strategy = "epoch",
    save_strategy = "no",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.05,
    logging_steps=1
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset ,
    eval_dataset=eval_dataset ,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [140]:
train_result = trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: assertion_type, sentence1, sentence2.
***** Running training *****
  Num examples = 6973
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1090


Epoch,Training Loss,Validation Loss,F1,Precision,Recall,Accuracy
1,0.257900,0.283527,0.524354,0.526741,0.524164,0.900000
2,0.377300,0.219978,0.582497,0.565831,0.607497,0.920000
3,0.066600,0.175833,0.740330,0.746506,0.740190,0.950000
4,0.134300,0.163414,0.727114,0.718615,0.737625,0.940000
5,0.196900,0.164050,0.740330,0.746506,0.740190,0.950000


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: assertion_type, sentence1, sentence2.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 32
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: assertion_type, sentence1, sentence2.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 32
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: assertion_type, sentence1, sentence2.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 32
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: assertion_type, sentence1, sentence2.
***** Running Evaluation *****
  Num examples = 10

In [142]:
metrics = train_result.metrics
metrics["train_samples"] = len(train_dataset)

trainer.save_model(f"models/{model_folder_name}")  # Saves the tokenizer too for easy upload

trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)

Saving model checkpoint to models/scibert-nli-ast-clf-1
Configuration saved in models/scibert-nli-ast-clf-1/config.json
Model weights saved in models/scibert-nli-ast-clf-1/pytorch_model.bin
tokenizer config file saved in models/scibert-nli-ast-clf-1/tokenizer_config.json
Special tokens file saved in models/scibert-nli-ast-clf-1/special_tokens_map.json


***** train metrics *****
  epoch                    =        5.0
  total_flos               =  1591638GF
  train_loss               =     0.2183
  train_runtime            = 0:06:59.08
  train_samples            =       6973
  train_samples_per_second =     83.193
  train_steps_per_second   =      2.601


In [143]:
print("*** Evaluate ***") 
metrics = trainer.evaluate(eval_dataset=eval_dataset)

metrics["eval_samples"] = len(eval_dataset)

trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: assertion_type, sentence1, sentence2.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 32


*** Evaluate ***
***** eval metrics *****
  epoch                   =        5.0
  eval_accuracy           =       0.95
  eval_f1                 =     0.7403
  eval_loss               =     0.1641
  eval_precision          =     0.7465
  eval_recall             =     0.7402
  eval_runtime            = 0:00:00.59
  eval_samples            =        100
  eval_samples_per_second =    168.916
  eval_steps_per_second   =      6.757


In [148]:
predictions, labels, _ = trainer.predict(eval_dataset, metric_key_prefix="predict")
predictions = np.argmax(predictions, axis=1)
print(classification_report(labels, predictions,target_names=label_list))

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: assertion_type, sentence1, sentence2.
***** Running Prediction *****
  Num examples = 100
  Batch size = 32


                              precision    recall  f1-score   support

                     present       0.96      0.98      0.97        65
                    possible       0.67      0.50      0.57         4
                      absent       1.00      0.96      0.98        23
                 conditional       0.00      0.00      0.00         1
                hypothetical       0.86      1.00      0.92         6
associated_with_someone_else       1.00      1.00      1.00         1

                    accuracy                           0.95       100
                   macro avg       0.75      0.74      0.74       100
                weighted avg       0.94      0.95      0.94       100



## Evaluate the model

In [151]:
# Local model
label_list = ['present',
 'possible',
 'absent',
 'conditional',
 'hypothetical',
 'associated_with_someone_else']

id2label = {i: label for i, label in enumerate(label_list)}
label2id = {v: k for k, v in id2label.items()}

# model_checkpoint = f"models/{model_folder_name}"
# model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, label2id=label2id, id2label=id2label)
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# effect_ner_model = pipeline(task="ner", model=model, tokenizer=tokenizer)


In [282]:
text_files = glob.glob(val_data_path + os.sep + txt_folder_name + os.sep +  "*.txt")
filename = ""
df = pd.DataFrame()
for file in tqdm(text_files):
    with open(file, 'r') as f:
        text = f.read()
        # split lines
        lines = text.split('\n')
        filename =[ file.split("/")[-1].split(".")[0]] * len(lines)
        df = df.append(pd.DataFrame({"text": lines, "filename": filename, "line_number": range(len(lines))}), ignore_index=True)

df = df.sort_values(by=["filename", "line_number"])
# remove empty text lines
# df = df[df.text != ""]
# # reset index
# df = df.reset_index(drop=True)

# add concepts
concept_df = []
for fname in tqdm(df["filename"].unique()):
    concept_dict = parse_concept(val_data_path + os.sep + concept_folder_name + os.sep + fname + ".con")
    for concept_text, start_line, start_word_number, end_line, end_word_number, concept_type in zip(*list(concept_dict.values())):
        if concept_type == "problem":
            text = df[(df["filename"] == fname) & (df["line_number"] == start_line-1)].text.values[0]
            concept_df.append({"filename": fname, "concept_text": concept_text, "text": text, "line_number":start_line})
            
concept_df = pd.DataFrame(concept_df)
# concept_df = concept_df.dropna(subset=["concept_text"])
df = concept_df[["filename", "line_number", "text", "concept_text"]]
df.rename(columns={"text":"sentence1", "concept_text":"sentence2"}, inplace=True)
df

100%|██████████| 128/128 [00:08<00:00, 14.29it/s]


,filename,line_number,sentence1,sentence2
0,0006,2,NVH,nvh
1,0006,5,"VT s/p cardiac cath , stent and amp ; amio loa...",vt
2,0006,33,SERIOUS INTERACTION :,serious interaction
3,0006,39,SERIOUS INTERACTION :,serious interaction
4,0006,45,SERIOUS INTERACTION :,serious interaction
...,...,...,...,...
6548,0475,105,She did very well over the course of her admis...,further bleeding
6549,0475,109,The serum albumin gradient was 1.8 consistent ...,portal hypertension
6550,0475,111,She therefore _________ criteria for a spontan...,a spontaneous bacterial peritonitis
6551,0475,112,Her platelet count stayed persistently low and...,low


In [283]:
predict_dataset = Dataset.from_pandas(df, preserve_index=False)
# predict_dataset = predict_dataset.select(range(10))
predict_dataset = predict_dataset.map(
                preprocess_function,
                batched=True,
                desc="Running tokenizer on prediction dataset",
            )
predict_dataset

Running tokenizer on prediction dataset:   0%|          | 0/7 [00:00<?, ?ba/s]

Dataset({
    features: ['filename', 'line_number', 'sentence1', 'sentence2', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6553
})

In [321]:
 predictions, labels, metrics = trainer.predict(predict_dataset, metric_key_prefix="predict")
 predictions = np.argmax(predictions, axis=1)
 len(predictions)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: filename, line_number, sentence1, sentence2.
***** Running Prediction *****
  Num examples = 6553
  Batch size = 2


6553

In [322]:
df["prediction"] = [label2ast[label] for label in predictions]
df

,filename,line_number,sentence1,sentence2,prediction
0,0006,2,NVH,nvh,present
1,0006,5,"VT s/p cardiac cath , stent and amp ; amio loa...",vt,present
2,0006,33,SERIOUS INTERACTION :,serious interaction,present
3,0006,39,SERIOUS INTERACTION :,serious interaction,present
4,0006,45,SERIOUS INTERACTION :,serious interaction,present
...,...,...,...,...,...
6548,0475,105,She did very well over the course of her admis...,further bleeding,absent
6549,0475,109,The serum albumin gradient was 1.8 consistent ...,portal hypertension,present
6550,0475,111,She therefore _________ criteria for a spontan...,a spontaneous bacterial peritonitis,present
6551,0475,112,Her platelet count stayed persistently low and...,low,present


In [323]:
df[df["filename"]=="0166"]

,filename,line_number,sentence1,sentence2,prediction
2161,0166,30,The patient was given printed instructions for...,back pain,present
2162,0166,32,back pain,back pain,present
2163,0166,36,ibuoprofen 800 every 6-8 hrs as needed skelaxi...,severe pain,hypothetical
2164,0166,40,"take medication as directed , return for sever...",severe or significantly worsening symptoms,hypothetical
2165,0166,40,"take medication as directed , return for sever...",weakness,hypothetical
2166,0166,40,"take medication as directed , return for sever...",loss of sensation work restrictions,hypothetical


In [329]:
import torch 
# empty cuda memory
torch.cuda.empty_cache()

In [324]:
# count predictions
df["prediction"].value_counts()

present                         4580
absent                          1414
possible                         262
hypothetical                     194
associated_with_someone_else      85
conditional                       18
Name: prediction, dtype: int64

In [325]:
final_df = df.copy()

In [326]:
# add concepts
for fname in tqdm(df["filename"].unique()):
    concept_dict = parse_concept(val_data_path + os.sep + concept_folder_name + os.sep + fname + ".con")
    for concept_text, start_line, start_word_number, end_line, end_word_number, concept_type in zip(*list(concept_dict.values())):
        if concept_type == "problem":
            df.loc[(df["line_number"] == start_line) & (df["filename"] == fname), "start_word_number"] = int(start_word_number)
            df.loc[(df["line_number"] == start_line) & (df["filename"] == fname), "end_word_number"] = int(end_word_number)
            df.loc[(df["line_number"] == start_line) & (df["filename"] == fname), "concept_type"] = concept_type
# change start_word_number to int type
df["start_word_number"] = df["start_word_number"].astype(int)
df["end_word_number"] = df["end_word_number"].astype(int)
df

100%|██████████| 127/127 [00:20<00:00,  6.32it/s]


,filename,line_number,sentence1,sentence2,prediction,start_word_number,end_word_number,concept_type
0,0006,2,NVH,nvh,present,0,0,problem
1,0006,5,"VT s/p cardiac cath , stent and amp ; amio loa...",vt,present,0,0,problem
2,0006,33,SERIOUS INTERACTION :,serious interaction,present,0,1,problem
3,0006,39,SERIOUS INTERACTION :,serious interaction,present,0,1,problem
4,0006,45,SERIOUS INTERACTION :,serious interaction,present,0,1,problem
...,...,...,...,...,...,...,...,...
6548,0475,105,She did very well over the course of her admis...,further bleeding,absent,14,15,problem
6549,0475,109,The serum albumin gradient was 1.8 consistent ...,portal hypertension,present,8,9,problem
6550,0475,111,She therefore _________ criteria for a spontan...,a spontaneous bacterial peritonitis,present,5,8,problem
6551,0475,112,Her platelet count stayed persistently low and...,low,present,5,5,problem


In [330]:
# for each file create <filename>.con
os.makedirs(val_data_path + os.sep + ast_folder_name, exist_ok=True)
# empty folder if exists
files = glob.glob(val_data_path + os.sep + ast_folder_name + os.sep + "*.con")
for file in files:
    os.remove(file)

for i, row in tqdm(df.iterrows()):
    filename = row["filename"]
    concept = row["sentence2"]
    line_num = row["line_number"]
    start_word_idx = row["start_word_number"]
    end_word_idx = row["end_word_number"]
    concept_type = row["concept_type"]
    pred_assertion = row["prediction"]
    with open(val_data_path + os.sep + ast_folder_name + os.sep + filename + ".ast", "a") as f:
        # fill like this c="bleeding risk" 126:11 126:12||t="problem"||a="present"
        f.write(
            f"c=\"{concept}\" {line_num}:{start_word_idx} {line_num}:{end_word_idx}||t=\"{concept_type}\"||a=\"{pred_assertion}\"\n"
        )
    


6553it [00:00, 9505.26it/s]


In [331]:
!zip -r scibert-val-ast.zip data/val/ast/

  adding: data/val/ast/ (stored 0%)
  adding: data/val/ast/0381.ast (deflated 75%)
  adding: data/val/ast/0467.ast (deflated 76%)
  adding: data/val/ast/0018.ast (deflated 78%)
  adding: data/val/ast/0206.ast (deflated 74%)
  adding: data/val/ast/0469.ast (deflated 73%)
  adding: data/val/ast/0273.ast (deflated 74%)
  adding: data/val/ast/0472.ast (deflated 74%)
  adding: data/val/ast/0342.ast (deflated 73%)
  adding: data/val/ast/0070.ast (deflated 70%)
  adding: data/val/ast/0066.ast (deflated 53%)
  adding: data/val/ast/0377.ast (deflated 74%)
  adding: data/val/ast/0222.ast (deflated 67%)
  adding: data/val/ast/0345.ast (deflated 73%)
  adding: data/val/ast/0405.ast (deflated 49%)
  adding: data/val/ast/0202.ast (deflated 70%)
  adding: data/val/ast/0474.ast (deflated 77%)
  adding: data/val/ast/0137.ast (deflated 67%)
  adding: data/val/ast/0077.ast (deflated 77%)
  adding: data/val/ast/0157.ast (deflated 60%)
  adding: data/val/ast/0278.ast (deflated 79%)
  adding: data/val/ast/0